# Applied Data Science Cap Stone Final Project Notebook

### In this project notebook I will be creating cluters and find the best location to open the biggest gaming centre in Toronto

#### To get started, importing all the required libraries

In [1]:
import numpy as np
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
!conda  install -c conda-forge folium
import folium

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.10.0              |             py_0          59 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be INSTAL

### 1. Getting the list of neighbourhoods in  Toronto extracting the list from wikipedia page.

### 2. Web scraping  by pandas html table scraping, pull tabular data directly from the web page into the data frame.

In [2]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(data, 'html.parser')

In [4]:
postalCodeList = []
boroughList = []
neighborhoodList = []

In [5]:
# Locate te table  and the postal code
soup.find('table').find_all('tr')

# find all the rows of the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [6]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

### View the DataFrame

In [7]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Delete the no-assigned columns

In [8]:
toronto_df_drop = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True)
toronto_df_drop.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Group neighbors in same borough

In [9]:
toronto_df_grouped = toronto_df_drop.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Neighborhood which is  "Not assigned", make the value the same as Borough

In [10]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Print the number of rows

In [11]:
toronto_df_grouped.shape

(103, 3)

### Getting the coordinates to utilise foursquare to pull list of venues near neighbourhood with the CSV file provided by the IBM.

In [12]:
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Renaming the postal code with postalcode

In [13]:
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)


### Now, merging the two DataFrames

In [14]:
toronto_df_new = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Getting the Latitudes adn Longitudes

In [15]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Exploring neighborhoods in Toronto

In [17]:
borough_names = list(toronto_df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

### Create a new DataFrame with only boroughs that contain the word Toronto because we are looking for a place to open a gaming centre in Toronto


In [18]:
toronto_df_new = toronto_df_new[toronto_df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(toronto_df_new.shape)
toronto_df_new.head()

(38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### Visualise Toronto map using folium package to verify the coordinates. 

In [59]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

### Use Foursquare to pull the venues, names, categories, latitudes, longitudes with the help of client ID and secret ID.

In [20]:
CLIENT_ID = '' 
CLIENT_SECRET = '' 
VERSION = '20180604'
LIMIT = 30
print('Your credentails are succesfully loaded and your ready to go vamshi, Good luck.')

Your credentails are succesfully loaded and your ready to go vamshi, Good luck.


In [67]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_df_new['Latitude'], toronto_df_new['Longitude'], toronto_df_new['PostalCode'], toronto_df_new['Borough'], toronto_df_new['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

### Convert venues list to new DataFrame

In [68]:
venues_df = pd.DataFrame(venues)


venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1705, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


### Check the unique categories from the venues.

In [69]:
venues_df['VenueCategory'].unique()[:200]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Greek Restaurant', 'Ice Cream Shop', 'Cosmetics Shop',
       'Italian Restaurant', 'Brewery', 'Yoga Studio', 'Restaurant',
       'Juice Bar', 'Fruit & Vegetable Store', 'Dessert Shop',
       'Pizza Place', 'Bookstore', 'Furniture / Home Store', 'Spa',
       'Grocery Store', 'Diner', 'Bubble Tea Shop', 'Coffee Shop',
       'Indian Restaurant', 'Caribbean Restaurant', 'Sports Bar',
       'Bakery', 'Café', 'American Restaurant', 'Sushi Restaurant',
       'Liquor Store', 'Gym', 'Burger Joint', 'Fish & Chips Shop', 'Park',
       'Pet Store', 'Burrito Place', 'Steakhouse', 'Movie Theater',
       'Fast Food Restaurant', 'Sandwich Place', 'Fish Market', 'Gay Bar',
       'Cheese Shop', 'Chinese Restaurant', 'Middle Eastern Restaurant',
       'Comfort Food Restaurant', 'Stationery Store',
       'Seafood Restaurant', 'Thai Restaurant', 'Coworking Space',
       'Gastropub', 'Latin American Restaurant', 'Bar',
       '

In [70]:
"Gaming Cafe" in venues_df['VenueCategory'].unique()

True

In [26]:
"Video Game Store" in venues_df['VenueCategory'].unique()

True

In [27]:
"Toy / Game Store" in venues_df['VenueCategory'].unique()

True

### Check how many venues are Returned

In [71]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",43,43,43,43,43,43
M4L,East Toronto,"The Beaches West, India Bazaar",18,18,18,18,18,18
M4M,East Toronto,Studio District,40,40,40,40,40,40
M4N,Central Toronto,Lawrence Park,4,4,4,4,4,4
M4P,Central Toronto,Davisville North,8,8,8,8,8,8
M4R,Central Toronto,North Toronto West,22,22,22,22,22,22
M4S,Central Toronto,Davisville,36,36,36,36,36,36
M4T,Central Toronto,"Moore Park, Summerhill East",2,2,2,2,2,2


### Analyzing each Area 

In [72]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1705, 234)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West, Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [73]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(38, 234)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.023256,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.023256
2,M4L,East Toronto,"The Beaches West, India Bazaar",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,M4M,East Toronto,Studio District,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.025000
4,M4N,Central Toronto,Lawrence Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,M4P,Central Toronto,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,M4R,Central Toronto,North Toronto West,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455
7,M4S,Central Toronto,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.027778,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000


### Getting the Number of venues in the neighborhood

In [74]:
len(toronto_grouped[toronto_grouped["Toy / Game Store"] > 0])

3

### NOTE: As we have very less number of gaming categories in the neighborhood, i thought of taking all related venues to gaming so,  its easy and more feasible to get the desired location, so I am going to check and find the clusters of all the related venues in the neighborhood.

In [37]:
len(toronto_grouped[toronto_grouped["Gaming Cafe"] > 0])

1

In [38]:
len(toronto_grouped[toronto_grouped["Video Game Store"] > 0])

3

### Grouping all venues together including neighborhoods

In [75]:
to_gaming = toronto_grouped[["Neighborhoods","Gaming Cafe","Toy / Game Store","Video Game Store"]]
to_gaming

,Neighborhoods,Gaming Cafe,Toy / Game Store,Video Game Store
0,The Beaches,0.00,0.000000,0.000000
1,"The Danforth West, Riverdale",0.00,0.000000,0.000000
2,"The Beaches West, India Bazaar",0.00,0.000000,0.000000
3,Studio District,0.00,0.000000,0.000000
4,Lawrence Park,0.00,0.000000,0.000000
5,Davisville North,0.00,0.000000,0.000000
6,North Toronto West,0.00,0.000000,0.000000
7,Davisville,0.00,0.027778,0.000000
8,"Moore Park, Summerhill East",0.00,0.000000,0.000000
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.00,0.000000,0.000000


## Clustering the Neighborhoods


### Running k- means an unsupervised machine learning algorithm to cluster the neighborhoods in Torornto into 3 clusters

In [76]:
toclusters = 3

to_clustering = to_gaming.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 0], dtype=int32)

In [77]:
to_merged = to_gaming.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [78]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged


,Neighborhood,Gaming Cafe,Toy / Game Store,Video Game Store,Cluster Labels
0,The Beaches,0.00,0.000000,0.000000,0
1,"The Danforth West, Riverdale",0.00,0.000000,0.000000,0
2,"The Beaches West, India Bazaar",0.00,0.000000,0.000000,0
3,Studio District,0.00,0.000000,0.000000,0
4,Lawrence Park,0.00,0.000000,0.000000,0
5,Davisville North,0.00,0.000000,0.000000,0
6,North Toronto West,0.00,0.000000,0.000000,0
7,Davisville,0.00,0.027778,0.000000,2
8,"Moore Park, Summerhill East",0.00,0.000000,0.000000,0
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.00,0.000000,0.000000,0


In [79]:
to_merged = to_merged.join(venues_df.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1705, 13)


,Neighborhood,Gaming Cafe,Toy / Game Store,Video Game Store,Cluster Labels,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,The Beaches,0.0,0.0,0.0,0,M4E,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
0,The Beaches,0.0,0.0,0.0,0,M4E,East Toronto,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
0,The Beaches,0.0,0.0,0.0,0,M4E,East Toronto,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
0,The Beaches,0.0,0.0,0.0,0,M4E,East Toronto,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
1,"The Danforth West, Riverdale",0.0,0.0,0.0,0,M4K,East Toronto,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [80]:
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged


(1705, 13)


,Neighborhood,Gaming Cafe,Toy / Game Store,Video Game Store,Cluster Labels,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,The Beaches,0.00,0.00,0.00,0,M4E,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
21,"Commerce Court, Victoria Hotel",0.00,0.00,0.00,0,M5L,Downtown Toronto,43.648198,-79.379817,John & Sons Oyster House,43.650656,-79.381613,Seafood Restaurant
21,"Commerce Court, Victoria Hotel",0.00,0.00,0.00,0,M5L,Downtown Toronto,43.648198,-79.379817,The Fairmont Royal York,43.645449,-79.381508,Hotel
21,"Commerce Court, Victoria Hotel",0.00,0.00,0.00,0,M5L,Downtown Toronto,43.648198,-79.379817,Hy's Steakhouse,43.649505,-79.382919,Steakhouse
21,"Commerce Court, Victoria Hotel",0.00,0.00,0.00,0,M5L,Downtown Toronto,43.648198,-79.379817,Petit Four Bakery,43.647744,-79.379588,Sandwich Place
21,"Commerce Court, Victoria Hotel",0.00,0.00,0.00,0,M5L,Downtown Toronto,43.648198,-79.379817,The Adelaide Hotel Toronto,43.649831,-79.380164,Hotel
21,"Commerce Court, Victoria Hotel",0.00,0.00,0.00,0,M5L,Downtown Toronto,43.648198,-79.379817,Marché Mövenpick,43.647262,-79.378600,Restaurant
21,"Commerce Court, Victoria Hotel",0.00,0.00,0.00,0,M5L,Downtown Toronto,43.648198,-79.379817,Design Exchange,43.647972,-79.380104,Art Gallery
21,"Commerce Court, Victoria Hotel",0.00,0.00,0.00,0,M5L,Downtown Toronto,43.648198,-79.379817,Dineen @CommerceCourt,43.648251,-79.380127,Coffee Shop
21,"Commerce Court, Victoria Hotel",0.00,0.00,0.00,0,M5L,Downtown Toronto,43.648198,-79.379817,Bymark,43.647217,-79.381252,Restaurant


In [81]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['BoroughLatitude'], to_merged['BoroughLongitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [82]:
map_clusters.save('map_clusters.html')
# saving the map as html

### Examine the Clusters

In [83]:
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Neighborhood,Gaming Cafe,Toy / Game Store,Video Game Store,Cluster Labels,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,The Beaches,0.0,0.0,0.0,0,M4E,East Toronto,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
21,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0,M5L,Downtown Toronto,43.648198,-79.379817,John & Sons Oyster House,43.650656,-79.381613,Seafood Restaurant
21,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0,M5L,Downtown Toronto,43.648198,-79.379817,The Fairmont Royal York,43.645449,-79.381508,Hotel
21,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0,M5L,Downtown Toronto,43.648198,-79.379817,Hy's Steakhouse,43.649505,-79.382919,Steakhouse
21,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0,M5L,Downtown Toronto,43.648198,-79.379817,Petit Four Bakery,43.647744,-79.379588,Sandwich Place
21,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0,M5L,Downtown Toronto,43.648198,-79.379817,The Adelaide Hotel Toronto,43.649831,-79.380164,Hotel
21,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0,M5L,Downtown Toronto,43.648198,-79.379817,Marché Mövenpick,43.647262,-79.378600,Restaurant
21,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0,M5L,Downtown Toronto,43.648198,-79.379817,Design Exchange,43.647972,-79.380104,Art Gallery
21,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0,M5L,Downtown Toronto,43.648198,-79.379817,Dineen @CommerceCourt,43.648251,-79.380127,Coffee Shop
21,"Commerce Court, Victoria Hotel",0.0,0.0,0.0,0,M5L,Downtown Toronto,43.648198,-79.379817,Bymark,43.647217,-79.381252,Restaurant


In [84]:
to_merged.loc[to_merged['Cluster Labels'] == 1]

,Neighborhood,Gaming Cafe,Toy / Game Store,Video Game Store,Cluster Labels,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
25,"Harbord, University of Toronto",0.0,0.0,0.028571,1,M5S,Downtown Toronto,43.662696,-79.400049,Second Cup Coffee Co.,43.665350,-79.398376,Café
25,"Harbord, University of Toronto",0.0,0.0,0.028571,1,M5S,Downtown Toronto,43.662696,-79.400049,Subway,43.664489,-79.399118,Sandwich Place
25,"Harbord, University of Toronto",0.0,0.0,0.028571,1,M5S,Downtown Toronto,43.662696,-79.400049,Second Cup,43.663551,-79.401787,Café
25,"Harbord, University of Toronto",0.0,0.0,0.028571,1,M5S,Downtown Toronto,43.662696,-79.400049,Red Fish Blue Fish Creative Cafe,43.662827,-79.402817,Café
25,"Harbord, University of Toronto",0.0,0.0,0.028571,1,M5S,Downtown Toronto,43.662696,-79.400049,A & C Games,43.664939,-79.403194,Video Game Store
25,"Harbord, University of Toronto",0.0,0.0,0.028571,1,M5S,Downtown Toronto,43.662696,-79.400049,Innis Town Hall,43.665420,-79.399546,College Arts Building
25,"Harbord, University of Toronto",0.0,0.0,0.028571,1,M5S,Downtown Toronto,43.662696,-79.400049,River Tai Restaurant,43.662902,-79.403167,Chinese Restaurant
25,"Harbord, University of Toronto",0.0,0.0,0.028571,1,M5S,Downtown Toronto,43.662696,-79.400049,Hart House Gym,43.664172,-79.394888,Gym
25,"Harbord, University of Toronto",0.0,0.0,0.028571,1,M5S,Downtown Toronto,43.662696,-79.400049,Coach House Printing,43.666320,-79.400277,Bookstore
25,"Harbord, University of Toronto",0.0,0.0,0.028571,1,M5S,Downtown Toronto,43.662696,-79.400049,Almond Butterfly,43.662836,-79.403365,Bakery


In [85]:
to_merged.loc[to_merged['Cluster Labels'] == 2]

,Neighborhood,Gaming Cafe,Toy / Game Store,Video Game Store,Cluster Labels,PostalCode,Borough,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
7,Davisville,0.00,0.027778,0.00,2,M4S,Central Toronto,43.704324,-79.388790,Jules Cafe Patisserie,43.704138,-79.388413,Dessert Shop
7,Davisville,0.00,0.027778,0.00,2,M4S,Central Toronto,43.704324,-79.388790,Marigold Indian Bistro,43.702881,-79.388008,Indian Restaurant
7,Davisville,0.00,0.027778,0.00,2,M4S,Central Toronto,43.704324,-79.388790,XO Gelato,43.705177,-79.388793,Dessert Shop
7,Davisville,0.00,0.027778,0.00,2,M4S,Central Toronto,43.704324,-79.388790,Starbucks,43.706084,-79.389355,Coffee Shop
7,Davisville,0.00,0.027778,0.00,2,M4S,Central Toronto,43.704324,-79.388790,GoodLife Fitness Toronto Mount Pleasant and Da...,43.700802,-79.386417,Gym
7,Davisville,0.00,0.027778,0.00,2,M4S,Central Toronto,43.704324,-79.388790,Zee Grill,43.704985,-79.388476,Seafood Restaurant
7,Davisville,0.00,0.027778,0.00,2,M4S,Central Toronto,43.704324,-79.388790,June Rowlands Park,43.700517,-79.389189,Park
7,Davisville,0.00,0.027778,0.00,2,M4S,Central Toronto,43.704324,-79.388790,Sakae Sushi,43.704944,-79.388704,Sushi Restaurant
7,Davisville,0.00,0.027778,0.00,2,M4S,Central Toronto,43.704324,-79.388790,Bread and Butter,43.701582,-79.387359,Sandwich Place
7,Davisville,0.00,0.027778,0.00,2,M4S,Central Toronto,43.704324,-79.388790,Florentia Ristorante,43.703594,-79.387985,Italian Restaurant


# Observation

### Most of Gaming related shops and cafes are in Cluster 2 which is around Ryerson, Garden District and Chinatown, Grange Park, Kensington Market. areas in lowest (close to zero) fall  in Cluster 1 areas which are Harbord, University of Toronto  nearby areas. Also, there are good opportunities to open near Central Bay street as the competition seems to be low. Looking at nearby venues, it seems Cluster 1 might be a good location as there are not a lot of Gaming Centres in these areas. Therefore, this project recommends the entrepreneur to open a gaming centre or an arcade in these locations with no competetion.


### NOTE : BUSINESS IDEA : As there are many gaming stores there will be a lot more advantages and business outcomes with collaborating with other game stores and cafes,  you can make your business a booming one. for an example, consider a group of people visited  your arcade to have some fun, they demand for an other game which you might not have but, its easy when you have a gaming stores nearby and it also gets easy to manage. 